# Prolog

In [ ]:
import pandas as pd
import seaborn as sb
import os
import ofwhpcparse as op
import datetime
import traceback
from scipy.optimize import curve_fit
import pathlib

In [ ]:
dfs = op.read_submissions()
dfs = op.derive_metrics(dfs)

In [ ]:
# Serialize forces we try to read in all even if most of them fail
_,_,fs = next(os.walk("submissions"))

dfsf = pd.DataFrame()
for fn in fs:
    if not fn.endswith("xlsm"):
        continue
    try:
        df_meta = pd.read_excel("submissions/" + fn, sheet_name="META Data")
        df_forces = pd.read_excel("submissions/" + fn, sheet_name="Aero Forces")
        dft=op.serialize_forces(df_forces, df_meta, fn)
    except Exception as e:
        print(f"failed force serialization {fn}")
        print(traceback.format_exc())
    dfsf = pd.concat([dfsf,dft])

In [ ]:
df_hardware = dfs[dfs["Track"] == "Hardware Track"]
df_software = dfs[dfs["Track"] == "Software Track"]

fig_folder = "figures_swtrack"
pathlib.Path(fig_folder).mkdir(parents=True, exist_ok=True)
doSaveFig = False  # Set to True to save figures

In [ ]:
# olympic swimming pool 50m*25m*2m 2.5e6 l water
V = 2.5e6 #l water
cp = 4.184  # kJ / kg⋅K
dT = 80 #K,
rho = 1 #kg/l
# 1kJ = 0.0002778 kWh
e = V * rho * dT * cp * 0.0002778

In [ ]:
dfs.columns

In [ ]:
df_software["Type"] = "Other"
df_software.loc[df_software["Contributor Affiliation"] == "Cineca", "Type"] = "Full GPU port"
df_software.loc[df_software["Contributor Affiliation"] == "Engys", "Type"] = "Full GPU port"
df_software.loc[df_software["Contributor Affiliation"] == "KIT/TUM", "Type"] = "GPU Plugin"
df_software_fine = df_software[df_software["Mesh"]=="fine"]
df_hardware_fine = df_hardware[df_hardware["Mesh"]=="fine"]


df_software.columns

# General

In [ ]:
# some statistics
print(f"""
total number of valid entries: {len(df_software)} / {len(dfs)},
Contributor ID: {set(df_software["Contributor ID"])}  ,
software track entries {len(df_software)} ,
processesed {len(set(dfs['CPU Model']))} different CPU vendors {set(dfs['CPU Model'])},
processesed {len(set(dfs['GPU Model']))} different GPUmodels {set(dfs['GPU Model'])},
-------------------
break-down by CPU model: {dfs["CPU Model"].value_counts()}
-------------------
-------------------
break-down by GPUmodel: {dfs["GPU Model"].value_counts()}
-------------------
max nodes {max(dfs['Number of Nodes'])},
max cores {max(dfs['Number of CPU Cores'])},
min wall clc {min(dfs['Run Wall-Clock Time [s]'])},
min time step software {min(df_software['Time per Iteration [s]'])},
min time step hardware {min(df_hardware['Time per Iteration [s]'])},
min energy software {min(df_software['Energy per Iteration [J]'])} [Ws],
min energy hardware {min(df_hardware['Energy per Iteration [J]'])} [Ws],
max FVOPS {max(df_software['FVOPS'])/1e6} [MFVOPS],
max FVOPS {max(df_hardware['FVOPS'])/1e6} [MFVOPS],
max energy {max(dfs['Energy-To-Solution [kWh]'])} [kWh],
total reported energy consumption {sum(dfs['Energy-To-Solution [kWh]'])} kWh, bringing {sum(dfs['Energy-To-Solution [kWh]'])/e} olympic swimming pools worth of water to a boil
decomposition methods {set(dfs["Decomposition Method"])},
renumbering methods {set(dfs["Renumbering Method"])}
network {set(dfs["Network Interconnect"])},
""")

In [ ]:
ax = sb.scatterplot(df_software,x="Number of CPU Cores", y="Run Wall-Clock Time [s]", hue="Contributor Affiliation", style="Mesh")
ax.set(xscale="log", yscale="log", title=" Software Track")
sb.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
fig = ax.get_figure()
op.save_fig(fig, fig_folder, "wct_cores_mesh", doSaveFig)

# FVOPS

In [ ]:
ax = sb.scatterplot(df_software,x="Number of Nodes", y="FVOPS", hue="Type", style="GPU Model")
ax.set(xscale="log", yscale="log", title=" Software Track")
sb.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
fig = ax.get_figure()
op.save_fig(fig, fig_folder, "FVOPS_Nodes_GPUOptimizationType", doSaveFig)

In [ ]:
ax = sb.scatterplot(df_software,x="Energy per Iteration [J]", y="FVOPS", hue="Software Optimization Category")
ax.set(xscale="log", yscale="log", title=" Software Track")
sb.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
fig = ax.get_figure()
op.save_fig(fig, fig_folder, "FVOPS_Energy_OptimizationCat", doSaveFig)

In [ ]:
ax = sb.scatterplot(df_software,x="Energy per Iteration [J]", y="FVOPS", hue="File Name")
ax.set(xscale="log", yscale="log", title=" Software Track")
sb.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
fig = ax.get_figure()
op.save_fig(fig, fig_folder, "FVOPS_EnergyPerIteration_FileName", doSaveFig)

In [ ]:
ax = sb.scatterplot(df_hardware,x="Energy per Iteration [J]", y="FVOPS", color="gray", marker="+")
ax = sb.scatterplot(df_software,x="Energy per Iteration [J]", y="FVOPS", hue="Software Optimization Category")
ax.set(xscale="log", yscale="log", title=" Software Track")
sb.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
fig = ax.get_figure()
op.save_fig(fig, fig_folder, "FVOPS_EnergyPerIteration_withHWTrack", doSaveFig)

In [ ]:
ax = sb.scatterplot(df_hardware_fine,x="Energy per Iteration [J]", y="FVOPS", color="gray", marker="+")
ax = sb.scatterplot(df_software_fine,x="Energy per Iteration [J]", y="FVOPS", hue="Software Optimization Category")
ax.set(xscale="log", yscale="log", title=" Software Track")
sb.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
fig = ax.get_figure()
op.save_fig(fig, fig_folder, "FVOPSEnergyPerIterationFine", doSaveFig)

# Scaling

In [ ]:
ax = sb.scatterplot(df_software,x="Number of CPU Cores", y="Run Wall-Clock Time [s]", hue="GPU Model", style="Mesh")
ax.set(xscale="log", yscale="log", title=" Software Track")
sb.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
fig = ax.get_figure()
op.save_fig(fig, fig_folder, "wct_cores_GPUModel", doSaveFig)

In [ ]:
df_software_filt = df_software
ax = sb.scatterplot(df_software_filt,x="Number of CPU Cores", y="Energy per Iteration [J]", hue="GPU Model", style="Mesh")
ax.set(xscale="log", yscale="log", title="Software Track")
sb.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
fig = ax.get_figure()
op.save_fig(fig, fig_folder, "EnergyPerIteration_cores_GPUModel", doSaveFig)

In [ ]:
ax = sb.scatterplot(df_software,x="Number of CPU Cores", y="Time per Iteration [s]", hue="GPU Model", style="Mesh")
ax.set(xscale="log", yscale="log", title=" Software Track")
sb.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
fig = ax.get_figure()
op.save_fig(fig, fig_folder, "TimePerIteration_cores_GPUModel", doSaveFig)

# Aerodynamic Coefficients

In [ ]:
dsfs_clean = dfsf.dropna()
dsfs_clean = dsfs_clean[dsfs_clean["Contributor Affiliation"] != "CFD FEA SERVICE SRL"]
dsfs_clean["Cd mean"] = "Cd"
dsfs_clean["Cl mean"] = "Cl"
dsfs_clean["Cs mean"] = "Cs"

sb.boxplot(
    dsfs_clean, x="cd_mean", y="Cd mean",
    whis=[0, 100], width=.6, palette="vlag"
)
sb.boxplot(
    dsfs_clean, x="cl_mean", y="Cl mean",
    whis=[0, 100], width=.6, palette="vlag"
)
sb.boxplot(
    dsfs_clean, x="cs_mean", y="Cs mean",
    whis=[0, 100], width=.6, palette="vlag"
)
sb.stripplot(dsfs_clean, x="cd_mean", y="Cd mean", size=0.5, color=".3")
sb.stripplot(dsfs_clean, x="cl_mean", y="Cl mean", size=0.5, color=".3")
ax = sb.stripplot(dsfs_clean, x="cs_mean", y="Cs mean", size=0.5, color=".3")

ax.set(xlabel='Mean value', ylabel='')

fig = ax.get_figure()

In [ ]:

sb.boxplot(
    dsfs_clean, x="cd_mean", y="Cd mean",
    whis=[0, 100], width=.6, palette="vlag"
)
ax = sb.stripplot(dsfs_clean, x="cd_mean", y="Cd mean", size=0.5, color=".3")

ax.set_xlabel('Mean value', fontsize=15)
ax.set_ylabel('')
ax.set_yticklabels(["Cd"], size = 15)
ax.set_xticklabels(ax.get_xticks(), size = 15)

fig = ax.get_figure()
fig.set_size_inches(6, 3)

In [ ]:
sb.boxplot(
    dsfs_clean, x="cl_mean", y="Cl mean",
    whis=[0, 100], width=.6, palette="vlag",
)
ax = sb.stripplot(dsfs_clean, x="cl_mean", y="Cl mean", size=0.5, color=".3")

ax.set_xlabel('Mean value', fontsize=15)
ax.set_ylabel('')
ax.set_yticklabels(["Cl"], size = 15)
ax.set_xticklabels(ax.get_xticks(), size = 15)
ax.set_xticks(ax.get_xticks()[::2]);            # show every 2th tick

fig = ax.get_figure()
fig.set_size_inches(6, 3)

In [ ]:
sb.boxplot(
    dsfs_clean, x="cs_mean", y="Cs mean",
    whis=[0, 100], width=.6, palette="vlag"
)
ax = sb.stripplot(dsfs_clean, x="cs_mean", y="Cs mean", size=0.5, color=".3")

ax.set_xlabel('Mean value', fontsize=15)
ax.set_ylabel('')
ax.set_yticklabels(["Cs"], size = 15)
ax.set_xticklabels(ax.get_xticks(), size = 15)

fig = ax.get_figure()
fig.set_size_inches(6, 3)